In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torch.nn.functional as F


import torchvision
from torchvision.models import resnet18, ResNet18_Weights, vgg16, VGG16_Weights
from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


import os
import sys
import copy
from torchsummary import summary


# from google.colab import drive
# drive.mount('/content/drive')
# #Code folder path
# %cd /content/drive/My Drive/ECE591_DL_CL_PROJECT/

#device config
torch.cuda.set_device(1)#Select GPU device 1
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Get Packages
sys.path.append('./Tools')

#Network architectures
from ResNet18_CIFAR10 import ResNet18_CIFAR10_Model
from ResNet18_TINYIMAGENET import ResNet18_TINYIMAGENET_Model
from VGG16_CIFAR10 import VGG16_CIFAR10_Model
from VGG16_TINYIMAGENET import VGG16_TINYIMAGENET_Model

#Non Corrupted DataLoaders
from CIFAR10_LOADER import CIFAR10DataLoader
from TINYIMAGENET_LOADER import TINYIMAGENETDataLoader

#Non Corrupted DataLoaders
from CIFAR10C_LOADER import CIFAR10C_DataLoader #Each Corruption is a entry in the dictionary
from TINYIMAGENET_C_LOADER import TinyImagenetC_DataLoader #Each Corruption is a entry in the dictionary

#Error Metrics
from ErrorMetrics import Evaluate_Model_TOP1
from GeneralPruning import iterative_pruning


# Unpruned Models

In [2]:
#Network weights
ResNet18_CIFAR10_Weights= torch.load("./Networks/ResNet18_CIFAR10_V0.pt")
ResNet18_TINYIMAGENET_Weights=torch.load("./Networks/ResNet18_TINY_IMAGENET_V0.pt")
VGG16_CIFAR10_Weights=torch.load("./Networks/VGG16_CIFAR10_V0.pt")
VGG16_TINYIMAGENET_Weights=torch.load("./Networks/VGG16_TINY_IMAGENET_V0.pt")

#Unpruned Models
ResNet18_CIFAR10_0P=ResNet18_CIFAR10_Model()
ResNet18_CIFAR10_0P.load_state_dict(ResNet18_CIFAR10_Weights)

ResNet18_TINYIMAGENET_0P=ResNet18_TINYIMAGENET_Model()
ResNet18_TINYIMAGENET_0P.load_state_dict(ResNet18_TINYIMAGENET_Weights)

VGG16_CIFAR10_0P=VGG16_CIFAR10_Model()
VGG16_CIFAR10_0P.load_state_dict(VGG16_CIFAR10_Weights)

VGG16_TINYIMAGENET_0P=VGG16_TINYIMAGENET_Model()
VGG16_TINYIMAGENET_0P.load_state_dict(VGG16_TINYIMAGENET_Weights)


<All keys matched successfully>

# Load Datasets

In [3]:
from unittest import loader

Times=5
trainloader_CIFAR10, testloader_CIFAR10=CIFAR10DataLoader(batch_size=512*Times) #Load Data for CIFAR10
loader_CIFAR10C=CIFAR10C_DataLoader(batch_size=512*Times) #Load Data for CIFAR10C

Times=1
trainloader_TINYIMAGENET, testloader_TINYIMAGENET=TINYIMAGENETDataLoader(NPZ=True,batch_size=512*Times) #Load Data for TINYIMAGENET
loader_TinyImagenetC=TinyImagenetC_DataLoader(NPZ=True,batch_size=512*Times) #Load Data for TINYIMAGENETC

Files already downloaded and verified
Files already downloaded and verified


# CIFAR 10

## Check Effect of Pruning Rate

In [4]:
#RestNet18
model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.5
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR_05PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.5 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.1369, Accuracy: 47769/50000 (95.54%)
Test set: Average loss: 0.3436, Accuracy: 8904/10000 (89.04%)

--Testing Corruptions--


100%|██████████| 19/19 [02:06<00:00,  6.65s/it]



Avg Corruption Accuracy: 76.10%
---------- Pruning Iteration:  1 / 7


Loss=0.16554157435894012 Batch_id=19 Accuracy=94.69: 100%|██████████| 20/20 [00:22<00:00,  1.12s/it]
Loss=0.1617097407579422 Batch_id=19 Accuracy=94.85: 100%|██████████| 20/20 [00:22<00:00,  1.15s/it] 
Loss=0.13211138546466827 Batch_id=19 Accuracy=95.06: 100%|██████████| 20/20 [00:22<00:00,  1.14s/it]
Loss=0.14149795472621918 Batch_id=19 Accuracy=95.06: 100%|██████████| 20/20 [00:23<00:00,  1.15s/it]
Loss=0.1460387259721756 Batch_id=19 Accuracy=95.41: 100%|██████████| 20/20 [00:22<00:00,  1.15s/it] 
Loss=0.12478166818618774 Batch_id=19 Accuracy=95.59: 100%|██████████| 20/20 [00:22<00:00,  1.14s/it]
Loss=0.10014083981513977 Batch_id=19 Accuracy=95.67: 100%|██████████| 20/20 [00:22<00:00,  1.14s/it]
Loss=0.11998182535171509 Batch_id=19 Accuracy=95.69: 100%|██████████| 20/20 [00:22<00:00,  1.14s/it]
Loss=0.12706376612186432 Batch_id=19 Accuracy=96.15: 100%|██████████| 20/20 [00:22<00:00,  1.15s/it]
Loss=0.11765690892934799 Batch_id=19 Accuracy=96.21: 100%|██████████| 20/20 [00:22<00:00,  

Epoch 00010: reducing learning rate of group 0 to 5.0000e-04.


...

Test set: Average loss: 0.4876, Accuracy: 8314/10000 (83.14%)

--Skipping Acc Comp For Corrupted Data--
Global Sparsity: 98.39% 

---------- Pruning Iteration:  7 / 7


Loss=1.1734293699264526 Batch_id=19 Accuracy=46.39: 100%|██████████| 20/20 [00:22<00:00,  1.11s/it]
Loss=0.9574352502822876 Batch_id=19 Accuracy=62.37: 100%|██████████| 20/20 [00:22<00:00,  1.11s/it]
Loss=0.8941705226898193 Batch_id=19 Accuracy=67.92: 100%|██████████| 20/20 [00:22<00:00,  1.11s/it]
Loss=0.877112865447998 Batch_id=19 Accuracy=70.57: 100%|██████████| 20/20 [00:22<00:00,  1.12s/it] 
Loss=0.7671022415161133 Batch_id=19 Accuracy=72.45: 100%|██████████| 20/20 [00:22<00:00,  1.11s/it]
Loss=0.7363071441650391 Batch_id=19 Accuracy=73.22: 100%|██████████| 20/20 [00:22<00:00,  1.11s/it]
Loss=0.7301596403121948 Batch_id=19 Accuracy=74.28: 100%|██████████| 20/20 [00:22<00:00,  1.10s/it]
Loss=0.7160676717758179 Batch_id=19 Accuracy=75.29: 100%|██████████| 20/20 [00:21<00:00,  1.10s/it]
Loss=0.7017037272453308 Batch_id=19 Accuracy=75.77: 100%|██████████| 20/20 [00:22<00:00,  1.11s/it]
Loss=0.6865518689155579 Batch_id=19 Accuracy=76.38: 100%|██████████| 20/20 [00:22<00:00,  1.12s/it]


Test set: Average loss: 0.6348, Accuracy: 7748/10000 (77.48%)

--Testing Corruptions--


100%|██████████| 19/19 [02:24<00:00,  7.61s/it]


Avg Corruption Accuracy: 62.02%
Global Sparsity: 99.17% 



In [5]:
#RestNet18
model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.25
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR_025PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.25 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.1380, Accuracy: 47752/50000 (95.50%)
Test set: Average loss: 0.3436, Accuracy: 8904/10000 (89.04%)

--Testing Corruptions--


100%|██████████| 19/19 [02:31<00:00,  7.98s/it]



Avg Corruption Accuracy: 76.10%
---------- Pruning Iteration:  1 / 16


Loss=0.17903752624988556 Batch_id=19 Accuracy=95.20: 100%|██████████| 20/20 [00:22<00:00,  1.12s/it]
Loss=0.14774319529533386 Batch_id=19 Accuracy=95.17: 100%|██████████| 20/20 [00:22<00:00,  1.12s/it]
Loss=0.13073188066482544 Batch_id=19 Accuracy=95.48: 100%|██████████| 20/20 [00:22<00:00,  1.13s/it]
Loss=0.12479492276906967 Batch_id=19 Accuracy=95.46: 100%|██████████| 20/20 [00:22<00:00,  1.13s/it]
Loss=0.14878182113170624 Batch_id=19 Accuracy=95.73: 100%|██████████| 20/20 [00:22<00:00,  1.12s/it]
Loss=0.11139100044965744 Batch_id=19 Accuracy=95.79: 100%|██████████| 20/20 [00:22<00:00,  1.12s/it]
Loss=0.12011612951755524 Batch_id=19 Accuracy=96.04: 100%|██████████| 20/20 [00:22<00:00,  1.12s/it]
Loss=0.10681983828544617 Batch_id=19 Accuracy=96.06: 100%|██████████| 20/20 [00:22<00:00,  1.12s/it]
Loss=0.12820295989513397 Batch_id=19 Accuracy=96.29: 100%|██████████| 20/20 [00:22<00:00,  1.12s/it]
Loss=0.1083822101354599 Batch_id=19 Accuracy=96.38: 100%|██████████| 20/20 [00:22<00:00,  1

Test set: Average loss: 0.3788, Accuracy: 8871/10000 (88.71%)

--Testing Corruptions--


...

Loss=0.6241158246994019 Batch_id=19 Accuracy=69.18: 100%|██████████| 20/20 [00:23<00:00,  1.18s/it]
Loss=0.5745075941085815 Batch_id=19 Accuracy=79.65: 100%|██████████| 20/20 [00:23<00:00,  1.18s/it]
Loss=0.49094507098197937 Batch_id=19 Accuracy=82.36: 100%|██████████| 20/20 [00:23<00:00,  1.17s/it]
Loss=0.5121439099311829 Batch_id=19 Accuracy=83.47: 100%|██████████| 20/20 [00:23<00:00,  1.17s/it] 
Loss=0.4644117057323456 Batch_id=19 Accuracy=83.98: 100%|██████████| 20/20 [00:23<00:00,  1.18s/it] 
Loss=0.45620423555374146 Batch_id=19 Accuracy=84.58: 100%|██████████| 20/20 [00:23<00:00,  1.18s/it]
Loss=0.39161866903305054 Batch_id=19 Accuracy=84.93: 100%|██████████| 20/20 [00:23<00:00,  1.17s/it]
Loss=0.4550611078739166 Batch_id=19 Accuracy=85.31: 100%|██████████| 20/20 [00:23<00:00,  1.18s/it] 
Loss=0.4497656524181366 Batch_id=19 Accuracy=85.28: 100%|██████████| 20/20 [00:23<00:00,  1.18s/it] 
Loss=0.430972695350647 Batch_id=19 Accuracy=85.78: 100%|██████████| 20/20 [00:23<00:00,  1.18

Epoch 00010: reducing learning rate of group 0 to 5.0000e-04.


Test set: Average loss: 0.4234, Accuracy: 8518/10000 (85.18%)

--Testing Corruptions--


100%|██████████| 19/19 [02:47<00:00,  8.79s/it]


Avg Corruption Accuracy: 69.55%
Global Sparsity: 98.95% 



In [6]:
#RestNet18
model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR_01PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.1380, Accuracy: 47744/50000 (95.49%)
Test set: Average loss: 0.3436, Accuracy: 8904/10000 (89.04%)

--Testing Corruptions--


100%|██████████| 19/19 [02:49<00:00,  8.91s/it]



Avg Corruption Accuracy: 76.10%
---------- Pruning Iteration:  1 / 44


Loss=0.1508515328168869 Batch_id=19 Accuracy=95.16: 100%|██████████| 20/20 [00:23<00:00,  1.19s/it] 
Loss=0.14574144780635834 Batch_id=19 Accuracy=95.26: 100%|██████████| 20/20 [00:23<00:00,  1.20s/it]
Loss=0.1322164386510849 Batch_id=19 Accuracy=95.39: 100%|██████████| 20/20 [00:23<00:00,  1.19s/it] 
Loss=0.15824085474014282 Batch_id=19 Accuracy=95.62: 100%|██████████| 20/20 [00:23<00:00,  1.20s/it]
Loss=0.1569252759218216 Batch_id=19 Accuracy=95.78: 100%|██████████| 20/20 [00:23<00:00,  1.19s/it] 
Loss=0.13301798701286316 Batch_id=19 Accuracy=95.82: 100%|██████████| 20/20 [00:23<00:00,  1.20s/it]


Epoch 00006: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00009: reducing learning rate of group 0 to 5.0000e-04.


Loss=0.21841639280319214 Batch_id=19 Accuracy=92.31: 100%|██████████| 20/20 [00:25<00:00,  1.26s/it]


Test set: Average loss: 0.3608, Accuracy: 8867/10000 (88.67%)

--Testing Corruptions--


100%|██████████| 19/19 [03:34<00:00, 11.29s/it]


Avg Corruption Accuracy: 72.60%
Global Sparsity: 98.98% 



In [7]:
#RestNet18
model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.05
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR_005PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.05 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.1387, Accuracy: 47726/50000 (95.45%)
Test set: Average loss: 0.3436, Accuracy: 8904/10000 (89.04%)

--Testing Corruptions--


100%|██████████| 19/19 [03:14<00:00, 10.21s/it]



Avg Corruption Accuracy: 76.10%
---------- Pruning Iteration:  1 / 90


Loss=0.17107811570167542 Batch_id=19 Accuracy=95.21: 100%|██████████| 20/20 [00:25<00:00,  1.27s/it]
Loss=0.1318756341934204 Batch_id=19 Accuracy=95.41: 100%|██████████| 20/20 [00:25<00:00,  1.30s/it] 
Loss=0.13986173272132874 Batch_id=19 Accuracy=95.56: 100%|██████████| 20/20 [00:25<00:00,  1.27s/it]
Loss=0.1305592954158783 Batch_id=19 Accuracy=95.47: 100%|██████████| 20/20 [00:25<00:00,  1.27s/it] 
Loss=0.12780627608299255 Batch_id=19 Accuracy=95.69: 100%|██████████| 20/20 [00:25<00:00,  1.26s/it]
Loss=0.11219228804111481 Batch_id=19 Accuracy=95.95: 100%|██████████| 20/20 [00:25<00:00,  1.27s/it]
Loss=0.09098128974437714 Batch_id=19 Accuracy=96.16: 100%|██████████| 20/20 [00:25<00:00,  1.28s/it]
Loss=0.13424579799175262 Batch_id=19 Accuracy=96.30: 100%|██████████| 20/20 [00:25<00:00,  1.28s/it]
Loss=0.12443310022354126 Batch_id=19 Accuracy=96.43: 100%|██████████| 20/20 [00:25<00:00,  1.27s/it]
Loss=0.1040455624461174 Batch_id=19 Accuracy=96.61: 100%|██████████| 20/20 [00:25<00:00,  1

Epoch 00010: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00008: reducing learning rate of group 0 to 5.0000e-04.


Loss=0.119663305580616 Batch_id=19 Accuracy=95.91: 100%|██████████| 20/20 [00:26<00:00,  1.32s/it]  
Loss=0.1146487146615982 Batch_id=19 Accuracy=96.16: 100%|██████████| 20/20 [00:26<00:00,  1.33s/it] 


Test set: Average loss: 0.3959, Accuracy: 8845/10000 (88.45%)

--Testing Corruptions--


100%|██████████| 19/19 [04:12<00:00, 13.27s/it]


Avg Corruption Accuracy: 72.98%
Global Sparsity: 98.97% 



In [8]:
#VGG16
model=copy.deepcopy(VGG16_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.5
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

SaveTrainStepsName="Progress/PruningIterations_VGG16_CIFAR_05PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.5 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: VGG16 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.1284, Accuracy: 47792/50000 (95.58%)
Test set: Average loss: 0.3662, Accuracy: 8909/10000 (89.09%)

--Testing Corruptions--


100%|██████████| 19/19 [03:45<00:00, 11.86s/it]



Avg Corruption Accuracy: 75.97%
---------- Pruning Iteration:  1 / 7


Loss=0.13468928635120392 Batch_id=19 Accuracy=95.19: 100%|██████████| 20/20 [00:26<00:00,  1.33s/it]
Loss=0.12698718905448914 Batch_id=19 Accuracy=95.42: 100%|██████████| 20/20 [00:26<00:00,  1.33s/it]
Loss=0.1572093516588211 Batch_id=19 Accuracy=95.38: 100%|██████████| 20/20 [00:26<00:00,  1.33s/it] 
Loss=0.13244730234146118 Batch_id=19 Accuracy=95.73: 100%|██████████| 20/20 [00:26<00:00,  1.32s/it]
Loss=0.1378818154335022 Batch_id=19 Accuracy=95.77: 100%|██████████| 20/20 [00:26<00:00,  1.32s/it] 


Epoch 00005: reducing learning rate of group 0 to 5.0000e-04.


...

Test set: Average loss: 0.3923, Accuracy: 8696/10000 (86.96%)

--Skipping Acc Comp For Corrupted Data--
Global Sparsity: 98.40% 

---------- Pruning Iteration:  7 / 7


Loss=0.7016785740852356 Batch_id=19 Accuracy=65.70: 100%|██████████| 20/20 [00:27<00:00,  1.37s/it]
Loss=0.5989462733268738 Batch_id=19 Accuracy=77.82: 100%|██████████| 20/20 [00:27<00:00,  1.37s/it]
Loss=0.5059518218040466 Batch_id=19 Accuracy=80.68: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it]
Loss=0.4944005608558655 Batch_id=19 Accuracy=82.43: 100%|██████████| 20/20 [00:27<00:00,  1.35s/it] 
Loss=0.4711568057537079 Batch_id=19 Accuracy=83.44: 100%|██████████| 20/20 [00:26<00:00,  1.35s/it] 
Loss=0.43439289927482605 Batch_id=19 Accuracy=84.06: 100%|██████████| 20/20 [00:27<00:00,  1.35s/it]
Loss=0.40651735663414 Batch_id=19 Accuracy=84.82: 100%|██████████| 20/20 [00:26<00:00,  1.35s/it]   
Loss=0.449518620967865 Batch_id=19 Accuracy=85.19: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it]  
Loss=0.42305198311805725 Batch_id=19 Accuracy=85.67: 100%|██████████| 20/20 [00:27<00:00,  1.35s/it]
Loss=0.39105528593063354 Batch_id=19 Accuracy=85.75: 100%|██████████| 20/20 [00:27<00:00,  1.3

Test set: Average loss: 0.4382, Accuracy: 8536/10000 (85.36%)

--Testing Corruptions--


100%|██████████| 19/19 [03:53<00:00, 12.29s/it]


Avg Corruption Accuracy: 70.78%
Global Sparsity: 99.18% 



In [9]:
#VGG16
model=copy.deepcopy(VGG16_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.25
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

SaveTrainStepsName="Progress/PruningIterations_VGG16_CIFAR_025PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.25 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: VGG16 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.1269, Accuracy: 47865/50000 (95.73%)
Test set: Average loss: 0.3662, Accuracy: 8909/10000 (89.09%)

--Testing Corruptions--


100%|██████████| 19/19 [04:10<00:00, 13.18s/it]



Avg Corruption Accuracy: 75.97%
---------- Pruning Iteration:  1 / 16


Loss=0.14221586287021637 Batch_id=19 Accuracy=95.43: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it]
Loss=0.13433930277824402 Batch_id=19 Accuracy=95.52: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it]
Loss=0.10105782747268677 Batch_id=19 Accuracy=95.79: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it]
Loss=0.09900466352701187 Batch_id=19 Accuracy=95.98: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it]
Loss=0.11268964409828186 Batch_id=19 Accuracy=96.09: 100%|██████████| 20/20 [00:27<00:00,  1.37s/it]
Loss=0.09931706637144089 Batch_id=19 Accuracy=96.35: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it]
Loss=0.11339118331670761 Batch_id=19 Accuracy=96.55: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it]


Epoch 00007: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00008: reducing learning rate of group 0 to 5.0000e-04.


Loss=0.18160022795200348 Batch_id=19 Accuracy=92.24: 100%|██████████| 20/20 [00:28<00:00,  1.41s/it]
Loss=0.20788158476352692 Batch_id=19 Accuracy=92.35: 100%|██████████| 20/20 [00:28<00:00,  1.41s/it]


Test set: Average loss: 0.3847, Accuracy: 8786/10000 (87.86%)

--Testing Corruptions--


100%|██████████| 19/19 [04:30<00:00, 14.23s/it]


Avg Corruption Accuracy: 73.10%
Global Sparsity: 98.96% 



In [10]:
#VGG16
model=copy.deepcopy(VGG16_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

SaveTrainStepsName="Progress/PruningIterations_VGG16_CIFAR_01PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: VGG16 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.1313, Accuracy: 47750/50000 (95.50%)
Test set: Average loss: 0.3662, Accuracy: 8909/10000 (89.09%)

--Testing Corruptions--


100%|██████████| 19/19 [04:17<00:00, 13.58s/it]



Avg Corruption Accuracy: 75.97%
---------- Pruning Iteration:  1 / 44


Loss=0.12203482538461685 Batch_id=19 Accuracy=95.56: 100%|██████████| 20/20 [00:28<00:00,  1.42s/it]
Loss=0.1117924153804779 Batch_id=19 Accuracy=95.64: 100%|██████████| 20/20 [00:28<00:00,  1.42s/it] 
Loss=0.12510855495929718 Batch_id=19 Accuracy=95.83: 100%|██████████| 20/20 [00:28<00:00,  1.43s/it]
Loss=0.1111983135342598 Batch_id=19 Accuracy=95.98: 100%|██████████| 20/20 [00:28<00:00,  1.43s/it] 
Loss=0.10918089002370834 Batch_id=19 Accuracy=96.21: 100%|██████████| 20/20 [00:28<00:00,  1.43s/it]
Loss=0.11554793268442154 Batch_id=19 Accuracy=96.40: 100%|██████████| 20/20 [00:28<00:00,  1.42s/it]
Loss=0.10601162910461426 Batch_id=19 Accuracy=96.55: 100%|██████████| 20/20 [00:28<00:00,  1.42s/it]
Loss=0.08023855835199356 Batch_id=19 Accuracy=96.64: 100%|██████████| 20/20 [00:28<00:00,  1.42s/it]
Loss=0.09962417185306549 Batch_id=19 Accuracy=96.88: 100%|██████████| 20/20 [00:28<00:00,  1.42s/it]
Loss=0.07574021071195602 Batch_id=19 Accuracy=97.03: 100%|██████████| 20/20 [00:28<00:00,  

Test set: Average loss: 0.4055, Accuracy: 8902/10000 (89.02%)

--Testing Corruptions--


...

Test set: Average loss: 0.4448, Accuracy: 8836/10000 (88.36%)

--Skipping Acc Comp For Corrupted Data--
Global Sparsity: 98.89% 

---------- Pruning Iteration:  44 / 44


Loss=0.17867496609687805 Batch_id=19 Accuracy=93.43: 100%|██████████| 20/20 [00:29<00:00,  1.50s/it]
Loss=0.14163091778755188 Batch_id=19 Accuracy=94.38: 100%|██████████| 20/20 [00:29<00:00,  1.50s/it]
Loss=0.1515614539384842 Batch_id=19 Accuracy=94.85: 100%|██████████| 20/20 [00:30<00:00,  1.50s/it] 
Loss=0.13548073172569275 Batch_id=19 Accuracy=95.24: 100%|██████████| 20/20 [00:30<00:00,  1.50s/it]
Loss=0.12958671152591705 Batch_id=19 Accuracy=95.28: 100%|██████████| 20/20 [00:29<00:00,  1.49s/it]
Loss=0.13504482805728912 Batch_id=19 Accuracy=95.65: 100%|██████████| 20/20 [00:29<00:00,  1.50s/it]
Loss=0.11671946942806244 Batch_id=19 Accuracy=95.68: 100%|██████████| 20/20 [00:29<00:00,  1.49s/it]
Loss=0.10774137079715729 Batch_id=19 Accuracy=95.73: 100%|██████████| 20/20 [00:29<00:00,  1.50s/it]
Loss=0.11082838475704193 Batch_id=19 Accuracy=95.95: 100%|██████████| 20/20 [00:29<00:00,  1.50s/it]
Loss=0.12470926344394684 Batch_id=19 Accuracy=95.90: 100%|██████████| 20/20 [00:29<00:00,  

Test set: Average loss: 0.4601, Accuracy: 8797/10000 (87.97%)

--Testing Corruptions--


100%|██████████| 19/19 [04:46<00:00, 15.08s/it]


Avg Corruption Accuracy: 73.48%
Global Sparsity: 99.00% 



# IMAGENET

In [11]:
#Choose a corruption level because simulation every level is too much
TinyImagenetC_CorruptionClasses=list(loader_TinyImagenetC.keys())
TinyImagenetC_CorruptionLevels=list(loader_TinyImagenetC[TinyImagenetC_CorruptionClasses[0]].keys()) #['1', '2', '3', '4', '5']

#Choose a level
clevel='1'
TiniyImageNetCLevel_Loader={}#Filtered for a specifig level of corruption
for cname in loader_TinyImagenetC.keys():
    TiniyImageNetCLevel_Loader[cname]=loader_TinyImagenetC[cname][clevel]

## Check Effect of Pruning Rate

In [12]:
#RestNet18
model=copy.deepcopy(ResNet18_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.5
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_ResNet18_IMAGENET_05PR_10Epochs_L2_0005"

ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.5 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)


--Get statistics before pruning--
Train set: Average loss: 0.5062, Accuracy: 87297/100000 (87.30%)
Test set: Average loss: 1.7581, Accuracy: 3018/5000 (60.36%)

--Testing Corruptions--


100%|██████████| 19/19 [03:55<00:00, 12.39s/it]



Avg Corruption Accuracy: 29.33%
---------- Pruning Iteration:  1 / 7


Loss=0.7331154942512512 Batch_id=195 Accuracy=80.67: 100%|██████████| 196/196 [01:23<00:00,  2.36it/s]
Loss=0.735856831073761 Batch_id=195 Accuracy=81.02: 100%|██████████| 196/196 [01:23<00:00,  2.34it/s] 
Loss=0.836553692817688 Batch_id=195 Accuracy=81.46: 100%|██████████| 196/196 [01:23<00:00,  2.34it/s] 
Loss=0.8680287599563599 Batch_id=195 Accuracy=81.83: 100%|██████████| 196/196 [01:23<00:00,  2.34it/s]


Epoch 00004: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00007: reducing learning rate of group 0 to 5.0000e-04.


Loss=1.909876823425293 Batch_id=195 Accuracy=49.12: 100%|██████████| 196/196 [01:25<00:00,  2.29it/s] 
Loss=2.021383762359619 Batch_id=195 Accuracy=49.58: 100%|██████████| 196/196 [01:25<00:00,  2.29it/s] 
Loss=2.1476054191589355 Batch_id=195 Accuracy=49.57: 100%|██████████| 196/196 [01:25<00:00,  2.29it/s]


Test set: Average loss: 2.1418, Accuracy: 2428/5000 (48.56%)

--Testing Corruptions--


100%|██████████| 19/19 [04:45<00:00, 15.02s/it]


Avg Corruption Accuracy: 20.91%
Global Sparsity: 98.32% 



In [13]:
#RestNet18
model=copy.deepcopy(ResNet18_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.25
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_ResNet18_IMAGENET_025PR_10Epochs_L2_0005"

ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.25 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.5064, Accuracy: 87185/100000 (87.19%)
Test set: Average loss: 1.7581, Accuracy: 3018/5000 (60.36%)

--Testing Corruptions--


100%|██████████| 19/19 [04:50<00:00, 15.27s/it]



Avg Corruption Accuracy: 29.33%
---------- Pruning Iteration:  1 / 16


Loss=0.6867067217826843 Batch_id=195 Accuracy=80.79: 100%|██████████| 196/196 [01:26<00:00,  2.27it/s]
Loss=0.7791337966918945 Batch_id=195 Accuracy=80.89: 100%|██████████| 196/196 [01:26<00:00,  2.26it/s]
Loss=0.925680935382843 Batch_id=195 Accuracy=81.37: 100%|██████████| 196/196 [01:26<00:00,  2.26it/s] 
Loss=0.7845808267593384 Batch_id=195 Accuracy=81.59: 100%|██████████| 196/196 [01:26<00:00,  2.26it/s]


Epoch 00004: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00007: reducing learning rate of group 0 to 5.0000e-04.


Loss=1.651384711265564 Batch_id=195 Accuracy=56.65: 100%|██████████| 196/196 [01:29<00:00,  2.20it/s] 
Loss=1.674770712852478 Batch_id=195 Accuracy=57.42: 100%|██████████| 196/196 [01:29<00:00,  2.20it/s] 
Loss=1.5158201456069946 Batch_id=195 Accuracy=57.40: 100%|██████████| 196/196 [01:28<00:00,  2.20it/s]


Test set: Average loss: 1.9630, Accuracy: 2674/5000 (53.48%)

--Testing Corruptions--


100%|██████████| 19/19 [05:49<00:00, 18.38s/it]


Avg Corruption Accuracy: 23.36%
Global Sparsity: 98.10% 



In [14]:
#RestNet18
model=copy.deepcopy(ResNet18_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_ResNet18_IMAGENET_01PR_10Epochs_L2_0005"

ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.5075, Accuracy: 87234/100000 (87.23%)
Test set: Average loss: 1.7581, Accuracy: 3018/5000 (60.36%)

--Testing Corruptions--


100%|██████████| 19/19 [05:52<00:00, 18.57s/it]



Avg Corruption Accuracy: 29.33%
---------- Pruning Iteration:  1 / 44


Loss=0.9555371999740601 Batch_id=195 Accuracy=80.32: 100%|██████████| 196/196 [01:29<00:00,  2.19it/s]
Loss=0.7646521329879761 Batch_id=195 Accuracy=80.44: 100%|██████████| 196/196 [01:29<00:00,  2.18it/s]
Loss=0.6305708289146423 Batch_id=195 Accuracy=80.77: 100%|██████████| 196/196 [01:30<00:00,  2.18it/s]
Loss=0.9158715009689331 Batch_id=195 Accuracy=81.34: 100%|██████████| 196/196 [01:30<00:00,  2.17it/s]
Loss=0.7348360419273376 Batch_id=195 Accuracy=81.46: 100%|██████████| 196/196 [01:29<00:00,  2.18it/s]
Loss=0.7509911060333252 Batch_id=195 Accuracy=81.76: 100%|██████████| 196/196 [01:30<00:00,  2.17it/s]


Epoch 00006: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00009: reducing learning rate of group 0 to 5.0000e-04.


Loss=1.5899970531463623 Batch_id=195 Accuracy=59.63: 100%|██████████| 196/196 [02:06<00:00,  1.55it/s]


Test set: Average loss: 1.9186, Accuracy: 2720/5000 (54.40%)

--Testing Corruptions--


100%|██████████| 19/19 [07:26<00:00, 23.50s/it]


Avg Corruption Accuracy: 25.05%
Global Sparsity: 98.13% 



In [15]:
#VGG16
model=copy.deepcopy(VGG16_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.5
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_VGG16_IMAGENET_05PR_10Epochs_L2_0005"

ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.5 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: VGG16 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.0133, Accuracy: 99812/100000 (99.81%)
Test set: Average loss: 2.7202, Accuracy: 2738/5000 (54.76%)

--Testing Corruptions--


100%|██████████| 19/19 [07:18<00:00, 23.06s/it]



Avg Corruption Accuracy: 28.70%
---------- Pruning Iteration:  1 / 7


Loss=0.05399547889828682 Batch_id=195 Accuracy=99.38: 100%|██████████| 196/196 [01:12<00:00,  2.69it/s] 
Loss=0.021267589181661606 Batch_id=195 Accuracy=99.55: 100%|██████████| 196/196 [01:12<00:00,  2.69it/s]
Loss=0.02902122773230076 Batch_id=195 Accuracy=99.63: 100%|██████████| 196/196 [01:12<00:00,  2.70it/s] 
Loss=0.01784689351916313 Batch_id=195 Accuracy=99.73: 100%|██████████| 196/196 [01:12<00:00,  2.69it/s] 
Loss=0.022539503872394562 Batch_id=195 Accuracy=99.71: 100%|██████████| 196/196 [01:12<00:00,  2.69it/s]
Loss=0.014312589541077614 Batch_id=195 Accuracy=99.76: 100%|██████████| 196/196 [01:12<00:00,  2.70it/s]
Loss=0.014446425251662731 Batch_id=195 Accuracy=99.78: 100%|██████████| 196/196 [01:12<00:00,  2.69it/s]
Loss=0.025498375296592712 Batch_id=195 Accuracy=99.78: 100%|██████████| 196/196 [01:12<00:00,  2.70it/s]
Loss=0.017466794699430466 Batch_id=195 Accuracy=99.81: 100%|██████████| 196/196 [01:13<00:00,  2.67it/s]


Epoch 00009: reducing learning rate of group 0 to 5.0000e-04.


...

Test set: Average loss: 2.3096, Accuracy: 2314/5000 (46.28%)

--Skipping Acc Comp For Corrupted Data--
Global Sparsity: 97.76% 

---------- Pruning Iteration:  7 / 7


Loss=2.454685926437378 Batch_id=195 Accuracy=40.01: 100%|██████████| 196/196 [01:13<00:00,  2.67it/s] 
Loss=2.2157163619995117 Batch_id=195 Accuracy=45.62: 100%|██████████| 196/196 [01:13<00:00,  2.67it/s]
Loss=2.0769782066345215 Batch_id=195 Accuracy=47.29: 100%|██████████| 196/196 [01:13<00:00,  2.66it/s]
Loss=2.2600724697113037 Batch_id=195 Accuracy=48.88: 100%|██████████| 196/196 [01:13<00:00,  2.66it/s]
Loss=2.2128405570983887 Batch_id=195 Accuracy=49.75: 100%|██████████| 196/196 [01:13<00:00,  2.67it/s]
Loss=1.8757374286651611 Batch_id=195 Accuracy=50.79: 100%|██████████| 196/196 [01:13<00:00,  2.67it/s]
Loss=2.250596523284912 Batch_id=195 Accuracy=50.97: 100%|██████████| 196/196 [01:13<00:00,  2.67it/s] 
Loss=1.69744074344635 Batch_id=195 Accuracy=51.74: 100%|██████████| 196/196 [01:13<00:00,  2.66it/s]  
Loss=1.6222190856933594 Batch_id=195 Accuracy=51.98: 100%|██████████| 196/196 [01:13<00:00,  2.68it/s]
Loss=1.8957191705703735 Batch_id=195 Accuracy=52.22: 100%|██████████| 196

Test set: Average loss: 2.3646, Accuracy: 2189/5000 (43.78%)

--Testing Corruptions--


100%|██████████| 19/19 [07:28<00:00, 23.58s/it]


Avg Corruption Accuracy: 19.76%
Global Sparsity: 98.53% 



In [16]:
#VGG16
model=copy.deepcopy(VGG16_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.25
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_VGG16_IMAGENET_025PR_10Epochs_L2_0005"

ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.25 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: VGG16 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.0139, Accuracy: 99788/100000 (99.79%)
Test set: Average loss: 2.7202, Accuracy: 2738/5000 (54.76%)

--Testing Corruptions--


100%|██████████| 19/19 [07:27<00:00, 23.54s/it]



Avg Corruption Accuracy: 28.70%
---------- Pruning Iteration:  1 / 16


Loss=0.015072214417159557 Batch_id=195 Accuracy=99.70: 100%|██████████| 196/196 [01:13<00:00,  2.67it/s]
Loss=0.01009774673730135 Batch_id=195 Accuracy=99.76: 100%|██████████| 196/196 [01:13<00:00,  2.65it/s] 
Loss=0.022131390869617462 Batch_id=195 Accuracy=99.75: 100%|██████████| 196/196 [01:13<00:00,  2.66it/s]
Loss=0.021602634340524673 Batch_id=195 Accuracy=99.81: 100%|██████████| 196/196 [01:13<00:00,  2.66it/s]
Loss=0.017704781144857407 Batch_id=195 Accuracy=99.83: 100%|██████████| 196/196 [01:14<00:00,  2.64it/s]


Epoch 00005: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00008: reducing learning rate of group 0 to 5.0000e-04.


Loss=1.4628822803497314 Batch_id=195 Accuracy=64.29: 100%|██████████| 196/196 [01:14<00:00,  2.65it/s]
Loss=1.3805153369903564 Batch_id=195 Accuracy=65.23: 100%|██████████| 196/196 [01:13<00:00,  2.65it/s]


Test set: Average loss: 2.1975, Accuracy: 2457/5000 (49.14%)

--Testing Corruptions--


100%|██████████| 19/19 [07:41<00:00, 24.30s/it]


Avg Corruption Accuracy: 22.67%
Global Sparsity: 98.31% 



In [17]:
#VGG16
model=copy.deepcopy(VGG16_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_VGG16_IMAGENET_01PR_10Epochs_L2_0005"

ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: VGG16 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                        lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.0138, Accuracy: 99794/100000 (99.79%)
Test set: Average loss: 2.7202, Accuracy: 2738/5000 (54.76%)

--Testing Corruptions--


100%|██████████| 19/19 [07:47<00:00, 24.59s/it]



Avg Corruption Accuracy: 28.70%
---------- Pruning Iteration:  1 / 44


Loss=0.01878349483013153 Batch_id=195 Accuracy=99.69: 100%|██████████| 196/196 [01:15<00:00,  2.59it/s] 
Loss=0.013316718861460686 Batch_id=195 Accuracy=99.73: 100%|██████████| 196/196 [01:15<00:00,  2.61it/s]
Loss=0.01344953291118145 Batch_id=195 Accuracy=99.76: 100%|██████████| 196/196 [01:15<00:00,  2.61it/s] 
Loss=0.011005342938005924 Batch_id=195 Accuracy=99.79: 100%|██████████| 196/196 [01:14<00:00,  2.64it/s]
Loss=0.006337241735309362 Batch_id=195 Accuracy=99.82: 100%|██████████| 196/196 [01:14<00:00,  2.63it/s]
Loss=0.009448068216443062 Batch_id=195 Accuracy=99.82: 100%|██████████| 196/196 [01:14<00:00,  2.63it/s]
Loss=0.01864456757903099 Batch_id=195 Accuracy=99.85: 100%|██████████| 196/196 [01:14<00:00,  2.63it/s] 
Loss=0.016627227887511253 Batch_id=195 Accuracy=99.85: 100%|██████████| 196/196 [01:14<00:00,  2.63it/s]


Epoch 00008: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00004: reducing learning rate of group 0 to 5.0000e-04.


Loss=1.1549475193023682 Batch_id=195 Accuracy=69.58: 100%|██████████| 196/196 [01:16<00:00,  2.55it/s]
Loss=1.133457899093628 Batch_id=195 Accuracy=71.14: 100%|██████████| 196/196 [01:17<00:00,  2.54it/s] 
Loss=1.0343396663665771 Batch_id=195 Accuracy=71.31: 100%|██████████| 196/196 [01:16<00:00,  2.55it/s]
Loss=1.0118805170059204 Batch_id=195 Accuracy=71.58: 100%|██████████| 196/196 [01:16<00:00,  2.55it/s]
Loss=0.9610400199890137 Batch_id=195 Accuracy=72.07: 100%|██████████| 196/196 [01:16<00:00,  2.55it/s]
Loss=1.1580064296722412 Batch_id=195 Accuracy=72.31: 100%|██████████| 196/196 [01:16<00:00,  2.56it/s]


Test set: Average loss: 2.2871, Accuracy: 2463/5000 (49.26%)

--Testing Corruptions--


100%|██████████| 19/19 [08:38<00:00, 27.30s/it]


Avg Corruption Accuracy: 21.47%
Global Sparsity: 98.35% 

